In [47]:
import numpy as np
import pandas as pd
import glob
import wget
from numba import njit
import os
import plotly.express as px
from datetime import datetime

In [2]:
pd.options.plotting.backend = "plotly"

Vamos começar com alguns conceitos antes de pegar um dataset bagunçado

## As perguntas

- Os gastos por ano
- Os gastos por mes
- Qual deputado gastou mais em cada legislatura
- Quais itens são os gastos principais dos deputados
- Quem é o deputado mais novo na hora de entrar
- Quem é o deputado mais velho na hora de entrar
- Qual deputado viajou mais do que todo mundo
- 

Primeiro vamos baixar os dados no website da camera do deputados no endereço [Dados Abertos da Camera dos Deputados](https://dadosabertos.camara.leg.br/)

In [3]:
folder_path = './data/deputados'


if not os.path.exists(folder_path):
    os.makedirs(folder_path)
    first_year = 2008
    last_year = datetime.now().year
    anos = range(first_year, last_year+1)

    for ano in anos:
        url = f'https://www.camara.leg.br/cotas/Ano-{str(ano)}.csv.zip'
        #!wget url -P './deputados/'
        wget.download(url, out=folder_path)

Para ler os datasets e concanetar todos

In [4]:
files = glob.glob("./data/deputados/*.csv.zip")
dfs = [pd.read_csv(f, compression='zip',sep=";") for f in files]

deps = pd.concat(dfs,ignore_index=True)

Vamos dar uma olhada nas caractristicas gerais do dataset final

In [5]:
deps.info(verbose=True, memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4066020 entries, 0 to 4066019
Data columns (total 31 columns):
 #   Column                     Dtype  
---  ------                     -----  
 0   txNomeParlamentar          object 
 1   cpf                        float64
 2   ideCadastro                float64
 3   nuCarteiraParlamentar      float64
 4   nuLegislatura              int64  
 5   sgUF                       object 
 6   sgPartido                  object 
 7   codLegislatura             int64  
 8   numSubCota                 int64  
 9   txtDescricao               object 
 10  numEspecificacaoSubCota    int64  
 11  txtDescricaoEspecificacao  object 
 12  txtFornecedor              object 
 13  txtCNPJCPF                 object 
 14  txtNumero                  object 
 15  indTipoDocumento           int64  
 16  datEmissao                 object 
 17  vlrDocumento               float64
 18  vlrGlosa                   float64
 19  vlrLiquido                 float64
 20  nu

In [6]:
deps.head()

,txNomeParlamentar,cpf,ideCadastro,nuCarteiraParlamentar,nuLegislatura,sgUF,sgPartido,codLegislatura,numSubCota,txtDescricao,...,numAno,numParcela,txtPassageiro,txtTrecho,numLote,numRessarcimento,vlrRestituicao,nuDeputadoId,ideDocumento,urlDocumento
0,LID.GOV-CD,NaN,NaN,NaN,2015,NaN,NaN,55,1,MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE ...,...,2017,0,NaN,NaN,1401321,NaN,NaN,2812,6328475,NaN
1,LID.GOV-CD,NaN,NaN,NaN,2015,NaN,NaN,55,1,MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE ...,...,2017,0,NaN,NaN,1436042,NaN,NaN,2812,6423547,NaN
2,LID.GOV-CD,NaN,NaN,NaN,2015,NaN,NaN,55,9,PASSAGEM AÉREA - REEMBOLSO,...,2017,0,MÁRCIA OLIVEIRA GOMES;,SÃO PAULO / BRASÍLIA,1401049,NaN,NaN,2812,6327499,NaN
3,LID.GOV-CD,NaN,NaN,NaN,2015,NaN,NaN,55,9,PASSAGEM AÉREA - REEMBOLSO,...,2017,0,MÁRCIA OLIVEIRA GOMES;,BRASÍLIA/SÃO PAULO,1401049,NaN,NaN,2812,6327494,NaN
4,LID.GOV-CD,NaN,NaN,NaN,2015,NaN,NaN,55,13,FORNECIMENTO DE ALIMENTAÇÃO DO PARLAMENTAR,...,2017,0,NaN,NaN,1411388,NaN,NaN,2812,6356388,NaN


In [7]:
deps.columns

Index(['txNomeParlamentar', 'cpf', 'ideCadastro', 'nuCarteiraParlamentar',
       'nuLegislatura', 'sgUF', 'sgPartido', 'codLegislatura', 'numSubCota',
       'txtDescricao', 'numEspecificacaoSubCota', 'txtDescricaoEspecificacao',
       'txtFornecedor', 'txtCNPJCPF', 'txtNumero', 'indTipoDocumento',
       'datEmissao', 'vlrDocumento', 'vlrGlosa', 'vlrLiquido', 'numMes',
       'numAno', 'numParcela', 'txtPassageiro', 'txtTrecho', 'numLote',
       'numRessarcimento', 'vlrRestituicao', 'nuDeputadoId', 'ideDocumento',
       'urlDocumento'],
      dtype='object')

In [8]:
deps.index

RangeIndex(start=0, stop=4066020, step=1)

In [9]:
len(deps['txNomeParlamentar'].unique())

1492

### Os gastos por ano

In [10]:
deps['datEmissao']

0          2017-07-03T00:00:00
1          2017-10-25T00:00:00
2          2017-07-03T00:00:00
3          2017-07-03T00:00:00
4          2017-08-08T00:00:00
                  ...         
4066015    2019-12-11T00:00:00
4066016    2019-12-31T00:00:00
4066017    2019-12-18T00:00:00
4066018                    NaN
4066019    2020-01-31T00:00:00
Name: datEmissao, Length: 4066020, dtype: object

In [12]:
deps[deps['datEmissao'].isnull()].shape

(198497, 31)

In [13]:
deps['datEmissao'] = pd.to_datetime(deps['datEmissao'])

In [14]:
deps['datEmissao'].min()

Timestamp('1959-11-03 00:00:00')

In [15]:
deps['datEmissao'].max()

Timestamp('2021-07-06 18:44:57')

In [16]:
deps[deps['datEmissao'].dt.year < 2008]['datEmissao']

849492    2001-06-14
849655    2001-03-12
909265    2001-02-07
909476    2001-06-03
917021    2001-05-10
             ...    
2767212   2004-12-14
2780402   2007-07-14
2799149   2001-07-27
2811370   2005-02-05
2838074   2001-03-15
Name: datEmissao, Length: 63, dtype: datetime64[ns]

Uma ideia para preencher os dados faltantes na coluna `datEmissao` podemos usar das colunas `numMes` e `numAno`

In [17]:
deps[['numAno', 'numMes']]

,numAno,numMes
0,2017,7
1,2017,10
2,2017,7
3,2017,7
4,2017,8
...,...,...
4066015,2019,12
4066016,2019,12
4066017,2019,12
4066018,2019,12


Vamos ver se tem alguma falha nos dados dessa duas colunas

In [22]:
deps[deps['numAno'].isnull()]

,txNomeParlamentar,cpf,ideCadastro,nuCarteiraParlamentar,nuLegislatura,sgUF,sgPartido,codLegislatura,numSubCota,txtDescricao,...,numAno,numParcela,txtPassageiro,txtTrecho,numLote,numRessarcimento,vlrRestituicao,nuDeputadoId,ideDocumento,urlDocumento


In [23]:
deps[deps['numMes'].isnull()]

,txNomeParlamentar,cpf,ideCadastro,nuCarteiraParlamentar,nuLegislatura,sgUF,sgPartido,codLegislatura,numSubCota,txtDescricao,...,numAno,numParcela,txtPassageiro,txtTrecho,numLote,numRessarcimento,vlrRestituicao,nuDeputadoId,ideDocumento,urlDocumento


In [24]:
pd.to_datetime(deps.loc[deps['datEmissao'].dt.year < 2008, 'numAno'].astype(str) + '/' + deps.loc[deps['datEmissao'].dt.year < 2008, 'numMes'].astype(str), format= '%Y/%-m')

849492    2011-06-01
849655    2011-03-01
909265    2011-03-01
909476    2011-06-01
917021    2011-04-01
             ...    
2767212   2010-11-01
2780402   2010-07-01
2799149   2010-07-01
2811370   2010-02-01
2838074   2010-03-01
Length: 63, dtype: datetime64[ns]

In [25]:
deps.loc[deps['datEmissao'].dt.year < 2008, 'datEmissao'] = pd.to_datetime(deps.loc[deps['datEmissao'].dt.year < 2008, 'numAno'].astype(str) +\
                                                                                     '/' + \
                                                                                    deps.loc[deps['datEmissao'].dt.year < 2008, 'numMes'].astype(str), format= '%Y/%m')

Agora vamos verificar se a operação deu certo

In [27]:
deps['datEmissao'].min()

Timestamp('2008-01-01 00:00:00')

In [28]:
deps['datEmissao'].max()

Timestamp('2021-07-06 18:44:57')

O proximo passo sera agrupar os dados para cada ano. 

In [30]:
deps.loc[:, ['numAno', 'vlrLiquido']].groupby(deps['numAno']).sum()

,numAno,vlrLiquido
numAno,,
2008,893560,2.118779e+05
2009,532780773,1.154050e+08
2010,701962350,1.543919e+08
2011,774574859,1.719673e+08
2012,720990140,1.704729e+08
2013,788145864,1.894656e+08
2014,717151162,1.964392e+08
2015,813256015,2.140812e+08
2016,772252992,2.210616e+08


In [32]:
deps.loc[:, ['numAno', 'vlrLiquido']].groupby(deps['numAno']).sum()['vlrLiquido'].plot()

### Os gastos por mes

In [33]:
deps.loc[:, ['numMes', 'vlrLiquido']].groupby(deps['numMes']).sum()['vlrLiquido'].plot()

### Qual deputado gastou mais em cada legislatura

In [34]:
deps['nuLegislatura'].unique()

array([2015, 2019, 2011, 2007])

In [37]:
deps2007 = deps.loc[deps['nuLegislatura'] == 2007, :]

In [38]:
deps2007.shape

(637283, 31)

In [39]:
deps2007[['txNomeParlamentar', 'vlrLiquido']].groupby('txNomeParlamentar').sum()

,vlrLiquido
txNomeParlamentar,
ABELARDO CAMARINHA,586511.46
ABELARDO LUPION,613722.33
ACÁCIO JÚNIOR,12912.52
ACÉLIO CASAGRANDE,282914.17
ADEMIR CAMILO,585212.99
...,...
ZÉ GERARDO,638146.59
ZÉ VIEIRA,692418.55
Átila Lins,637825.29


In [49]:
gastos = deps2007[['txNomeParlamentar', 'vlrLiquido']].groupby('txNomeParlamentar').sum()['vlrLiquido'].sort_values(ascending=False)
gastos

txNomeParlamentar
Raul Henry                       807175.61
SABINO CASTELO BRANCO            794180.98
Wellington Roberto               782567.12
VICENTE ARRUDA                   764207.87
ANTONIO CARLOS MAGALHÃES NETO    749993.03
                                   ...    
ROSANE FERREIRA                      70.00
DJALMA BERGER                         1.04
JOÃO FONTES                        -149.62
ATHOS AVELINO                      -334.42
ELIANE ROLIM                      -1137.24
Name: vlrLiquido, Length: 644, dtype: float64

In [50]:
px.bar(x = gastos.index[:10], y = gastos.values[:10])

Agora vamos ver com o que ele gastou as sua cota

In [53]:
deps[(deps['txNomeParlamentar'] == 'Raul Henry') & (deps['nuLegislatura'] == 2007)].groupby('txtDescricao').sum()['vlrLiquido'].sort_values()

txtDescricao
LOCOMOÇÃO, ALIMENTAÇÃO E  HOSPEDAGEM                                31.53
PASSAGEM AÉREA - REEMBOLSO                                         173.00
AQUISIÇÃO DE MATERIAL DE ESCRITÓRIO.                               318.90
AQUISIÇÃO OU LOC. DE SOFTWARE; SERV. POSTAIS; ASS.                 746.50
ASSINATURA DE PUBLICAÇÕES                                         1478.00
LOCAÇÃO DE VEÍCULOS AUTOMOTORES OU FRETAMENTO DE EMBARCAÇÕES      2613.00
FORNECIMENTO DE ALIMENTAÇÃO DO PARLAMENTAR                        2684.47
HOSPEDAGEM ,EXCETO DO PARLAMENTAR NO DISTRITO FEDERAL.            3651.60
MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE PARLAMENTAR        27347.47
SERVIÇOS POSTAIS                                                 32193.38
TELEFONIA                                                        76073.71
COMBUSTÍVEIS E LUBRIFICANTES.                                    85509.97
DIVULGAÇÃO DA ATIVIDADE PARLAMENTAR.                             88500.00
PASSAGEM AÉREA - RPA     

### Quais itens são os gastos principais dos deputados

In [56]:
deps2007.groupby(['txtDescricao']).sum()['vlrLiquido'].sort_values()

txtDescricao
ASSINATURA DE PUBLICAÇÕES                                         323692.56
AQUISIÇÃO DE MATERIAL DE ESCRITÓRIO.                              546967.03
AQUISIÇÃO OU LOC. DE SOFTWARE; SERV. POSTAIS; ASS.                951284.94
SERVIÇO DE SEGURANÇA PRESTADO POR EMPRESA ESPECIALIZADA.         2011735.57
FORNECIMENTO DE ALIMENTAÇÃO DO PARLAMENTAR                       2381273.58
HOSPEDAGEM ,EXCETO DO PARLAMENTAR NO DISTRITO FEDERAL.           2943778.68
LOCOMOÇÃO, ALIMENTAÇÃO E  HOSPEDAGEM                             5439018.26
SERVIÇOS POSTAIS                                                12701231.60
PASSAGEM AÉREA - REEMBOLSO                                      14739989.58
CONSULTORIAS, PESQUISAS E TRABALHOS TÉCNICOS.                   27402720.62
MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE PARLAMENTAR       28367250.81
PASSAGEM AÉREA - RPA                                            29193738.55
LOCAÇÃO DE VEÍCULOS AUTOMOTORES OU FRETAMENTO DE EMBARCAÇÕES    31123491.66

### Quem é o deputado mais novo na hora de entrar

Para achar a idade de cada deputado precisamos de outro dataset que contenha essa informação e depois temos que fazer um cruzamento de dados

In [58]:
deps_info = pd.read_csv('./data/deputados/deputados_profissão.csv', sep=';')

deps_info.head()

,uri,nome,idLegislaturaInicial,idLegislaturaFinal,nomeCivil,cpf,siglaSexo,urlRedeSocial,urlWebsite,dataNascimento,dataFalecimento,ufNascimento,municipioNascimento
0,https://dadosabertos.camara.leg.br/api/v2/depu...,LOPES GAMA,1,2,Caetano Maria Lopes Gama,NaN,M,NaN,NaN,1795-08-05,1864-06-21,PE,Recife
1,https://dadosabertos.camara.leg.br/api/v2/depu...,CALMON,1,4,Miguel Calmon Du Pin e Almeida,NaN,M,NaN,NaN,1796-12-22,1865-09-13,BA,Santo Amaro
2,https://dadosabertos.camara.leg.br/api/v2/depu...,MONTEZUMA,2,8,Francisco Gê Acayaba Montezuma,NaN,M,NaN,NaN,1794-03-23,1870-02-15,BA,Salvador
3,https://dadosabertos.camara.leg.br/api/v2/depu...,ALBUQUERQUE CAVALCANTI,3,4,José Mariano de Albuquerque Cavalcanti,NaN,M,NaN,NaN,1772-05-20,1844-08-20,CE,NaN
4,https://dadosabertos.camara.leg.br/api/v2/depu...,CARNEIRO DA CUNHA,2,8,Joaquim Manoel Carneiro da Cunha,NaN,M,NaN,NaN,1796-01-01,1859-09-07,NaN,NaN


Prestem atenção na coluna `nomeCivil` e comparam com a coluna `txNomeParlamentar` do dataset `deps`. Os nomes tem formato diferente. Então, temos que ter cuidado na hora de fazer o cruzamento

In [86]:
deps_info['nomeCivil'] = deps_info['nomeCivil'].str.upper() 
deps_info['nome'] = deps_info['nome'].str.upper() 
deps['txNomeParlamentar'] = deps['txNomeParlamentar'].str.upper() 

In [107]:
deps_idade1 = pd.merge(left=deps[['txNomeParlamentar', 'nuLegislatura']], right=deps_info[['nomeCivil', 'dataNascimento', 'nome']], left_on = 'txNomeParlamentar', right_on='nomeCivil')

In [108]:
deps_idade1.head()

,txNomeParlamentar,nuLegislatura,nomeCivil,dataNascimento,nome
0,CELSO PANSERA,2015,CELSO PANSERA,1963-10-10,CELSO PANSERA
1,CELSO PANSERA,2015,CELSO PANSERA,1963-10-10,CELSO PANSERA
2,CELSO PANSERA,2015,CELSO PANSERA,1963-10-10,CELSO PANSERA
3,CELSO PANSERA,2015,CELSO PANSERA,1963-10-10,CELSO PANSERA
4,CELSO PANSERA,2015,CELSO PANSERA,1963-10-10,CELSO PANSERA


In [119]:
deps_idade1.shape

(399123, 5)

In [120]:
deps_idade1.drop_duplicates('nomeCivil', ignore_index=True, inplace=True)

In [121]:
nomes1 = deps_idade1['txNomeParlamentar'].unique()
nomes1

array(['CELSO PANSERA', 'SIMÃO SESSIM', 'ARNALDO FARIA DE SÁ',
       'RUBENS BUENO', 'LUIS CARLOS HEINZE', 'IVAN VALENTE',
       'JORGE TADEU MUDALEN', 'NELSON MARQUEZELLI', 'HERMES PARCIANELLO',
       'NELSON MEURER', 'REINHOLD STEPHANES', 'VALDIR COLATTO',
       'LUIZ COUTO', 'RAIMUNDO GOMES DE MATOS', 'JOSUÉ BENGTSON',
       'SILAS CÂMARA', 'JOSÉ OTÁVIO GERMANO', 'AROLDE DE OLIVEIRA',
       'JANDIRA FEGHALI', 'MIRO TEIXEIRA', 'RODRIGO GARCIA',
       'VALMIR PRASCIDELLI', 'MARCELO SQUASSONI', 'MAURO PEREIRA',
       'CELSO MALDANER', 'MAURO MARIANI', 'PAULO ABI-ACKEL',
       'PAULO PEREIRA DA SILVA', 'VANDERLEI MACRIS',
       'DANRLEI DE DEUS HINTERHOLZ', 'NELSON PADOVANI', 'JOÃO RODRIGUES',
       'SERGIO ZVEITER', 'GIOVANI CHERINI', 'LUIZ CARLOS RAMOS',
       'RONALDO CARLETTO', 'EVAIR VIEIRA DE MELO', 'GIUSEPPE VECCI',
       'CHRISTIANE DE SOUZA YARED', 'LUCIANO DUCCI', 'MIGUEL LOMBARDI',
       'VITOR LIPPI', 'ANGELA ALBINO', 'DARCI DE MATOS', 'NERI GELLER',
       'MA

In [122]:
len(nomes1)

109

Mas isso é muito pouco! Vamos ver o que está acontecendo

In [117]:
deps[~deps['txNomeParlamentar'].isin(nomes1)]['txNomeParlamentar'].unique()

array(['LID.GOV-CD', 'LID.GOV-CN',
       'LIDERANÇA DO PARTIDO REPUBLICANO DA ORDEM SOCIAL', ...,
       'CORONEL BITTENCOURT', 'CARLA STEPHANINI', 'RICARDO PERICAR'],
      dtype=object)

In [82]:
deps_info[deps_info['nome'].str.contains('CORONEL BITTENCOURT')]

,uri,nome,idLegislaturaInicial,idLegislaturaFinal,nomeCivil,cpf,siglaSexo,urlRedeSocial,urlWebsite,dataNascimento,dataFalecimento,ufNascimento,municipioNascimento
7555,https://dadosabertos.camara.leg.br/api/v2/depu...,CORONEL BITTENCOURT,55,55,ISAIAS FERREIRA BITTENCOURT,NaN,M,NaN,NaN,1966-10-17,NaN,MS,Terenos


Aparentemente a coluna `txNomeParlamentar` não tem os nomes civis e em alguns casos tem apelidos. Então vamos fazer outro cruzamento com essa coluna

In [110]:
deps_idade2 = pd.merge(left=deps[['txNomeParlamentar', 'nuLegislatura']], right=deps_info[['nomeCivil', 'dataNascimento', 'nome']], left_on = 'txNomeParlamentar', right_on='nome')

In [126]:
nomes2 = deps_idade2['txNomeParlamentar'].unique()


In [127]:
len(nomes2)

1454

In [128]:
deps_idade2.shape

(1454, 5)

In [129]:
deps_idade2.drop_duplicates('nome', ignore_index=True, inplace=True)

Agora vamos concatenar os dois dataset de `deps_idade`s

In [124]:
deps_idade = pd.concat([deps_idade1, deps_idade2])

In [125]:
deps_idade.shape

(1563, 5)

In [133]:
nomes = np.concatenate([nomes1, nomes2])

In [135]:
len(nomes)

1563

In [136]:
deps[~deps['txNomeParlamentar'].isin(nomes)]['txNomeParlamentar'].unique()

array(['LID.GOV-CD', 'LID.GOV-CN',
       'LIDERANÇA DO PARTIDO REPUBLICANO DA ORDEM SOCIAL',
       'LIDERANÇA DO PDT', 'LIDERANÇA DO PHS', 'LIDERANÇA DO PODEMOS',
       'LIDERANÇA DO PSDB', 'LIDERANÇA DO PSOL', 'LIDERANÇA DO PT',
       'LIDERANÇA DO PTN', 'LIDERANÇA DO SOLIDARIEDADE',
       'LIDERANÇA DO CIDADANIA', 'LIDERANÇA DO DEMOCRATAS',
       'LIDERANÇA DO NOVO', 'LIDERANÇA DO PROGRESSISTAS',
       'LIDERANÇA DO PSD', 'LIDERANÇA DO PSL', 'LIDERANÇA DO PTB',
       'LIDMIN', 'IRIS SIMÕES', 'RÉGIS DE OLIVEIRA',
       'JOSE EDUARDO CARDOZO', 'NILSON MOURAO', 'EUNICIO OLIVEIRA',
       'LÉO ALCÂNTARA', 'THEMISTOCLES SAMPAIO', 'VIRGILIO GUIMARAES',
       'JOSÉ SANTANA VASCONCELLOS', 'CLAUDIO DIAZ',
       'JOSÉ FERNANDO APARECIDO DE OLI', 'SÉRGIO PETECÃO', 'DR.TALMIR',
       'VITAL DO REGO FILHO', 'OSORIO ADRIANO', 'JERONIMO REIS',
       'LIDERANÇA DO AVANTE'], dtype=object)

Vixi, ainda tem gente sem data de nascimento. Vamos ver quem são eles

In [139]:
deps_info[(deps_info['nomeCivil'] == 'OSORIO ADRIANO') | (deps_info['nome'] == 'OSORIO ADRIANO')]

,uri,nome,idLegislaturaInicial,idLegislaturaFinal,nomeCivil,cpf,siglaSexo,urlRedeSocial,urlWebsite,dataNascimento,dataFalecimento,ufNascimento,municipioNascimento


In [140]:
deps_info[(deps_info['nomeCivil'] == 'EUNICIO OLIVEIRA') | (deps_info['nome'] == 'EUNICIO OLIVEIRA')]

,uri,nome,idLegislaturaInicial,idLegislaturaFinal,nomeCivil,cpf,siglaSexo,urlRedeSocial,urlWebsite,dataNascimento,dataFalecimento,ufNascimento,municipioNascimento


In [147]:
deps_info[deps_info['nomeCivil'].str.contains('OSORIO') ]

,uri,nome,idLegislaturaInicial,idLegislaturaFinal,nomeCivil,cpf,siglaSexo,urlRedeSocial,urlWebsite,dataNascimento,dataFalecimento,ufNascimento,municipioNascimento
916,https://dadosabertos.camara.leg.br/api/v2/depu...,MIRANDA OSÓRIO,15,15,MANOEL PINHEIRO DE MIRANDA OSORIO,NaN,M,NaN,NaN,NaN,NaN,NaN,NaN
947,https://dadosabertos.camara.leg.br/api/v2/depu...,FERNANDO OSÓRIO,16,17,FERNANDO LUIZ OSORIO,NaN,M,NaN,NaN,1848-05-30,NaN,RS,Bagé
1347,https://dadosabertos.camara.leg.br/api/v2/depu...,ROCHA OSORIO,22,22,MANOEL LUIZ DA ROCHA OSORIO,NaN,M,NaN,NaN,NaN,NaN,NaN,NaN
2471,https://dadosabertos.camara.leg.br/api/v2/depu...,OSÓRIO DE PAIVA,30,31,VICENTE OSORIO DE PAIVA,NaN,M,NaN,NaN,1849-08-21,NaN,CE,Fortaleza
6250,https://dadosabertos.camara.leg.br/api/v2/depu...,OSORIO SANTA CRUZ,49,49,OSORIO LEÃO SANTA CRUZ,NaN,M,NaN,NaN,1941-08-21,NaN,GO,Rio Verde


**Alguem tem que avisar a equipe de dados da camera de deputados para corrigir esse erro!**

Para calcular a idade de cada parlamentar no ano da sua eleição precisamos transformar a coluna `nuLegislatura` em tipo `datetime`

In [151]:
deps_idade['nuLegislatura'] = pd.to_datetime(deps_idade['nuLegislatura'], format='%Y')
deps_idade['dataNascimento'] = pd.to_datetime(deps_idade['dataNascimento'], format='%Y-%m-%d')

In [152]:
deps_idade['idade'] = deps_idade['nuLegislatura']  - deps_idade['dataNascimento'] 

In [153]:
deps_idade

,txNomeParlamentar,nuLegislatura,nomeCivil,dataNascimento,nome,idade
0,CELSO PANSERA,2015-01-01,CELSO PANSERA,1963-10-10,CELSO PANSERA,18711 days
1,SIMÃO SESSIM,2015-01-01,SIMÃO SESSIM,1935-12-08,SIMÃO SESSIM,28879 days
2,ARNALDO FARIA DE SÁ,2015-01-01,ARNALDO FARIA DE SÁ,1945-12-30,ARNALDO FARIA DE SÁ,25204 days
3,RUBENS BUENO,2015-01-01,RUBENS BUENO,1948-05-23,RUBENS BUENO,24329 days
4,LUIS CARLOS HEINZE,2015-01-01,LUIS CARLOS HEINZE,1950-09-14,LUIS CARLOS HEINZE,23485 days
...,...,...,...,...,...,...
1449,MARFIZA GALVÃO,2015-01-01,MARFIZA DE LIMA GALVÃO,1979-03-30,MARFIZA GALVÃO,13061 days
1450,GUSTAVO MITRE,2015-01-01,GUSTAVO MARQUES CARVALHO MITRE,1966-12-22,GUSTAVO MITRE,17542 days
1451,CORONEL BITTENCOURT,2015-01-01,ISAIAS FERREIRA BITTENCOURT,1966-10-17,CORONEL BITTENCOURT,17608 days
1452,CARLA STEPHANINI,2015-01-01,CARLA CHARBEL STEPHANINI,1964-10-27,CARLA STEPHANINI,18328 days


In [158]:
deps_idade['idade'].min()

Timedelta('7691 days 00:00:00')

In [159]:
deps_idade['idade'].min()/pd.Timedelta(days=365)

21.07123287671233

In [160]:
deps_idade['idade'].argmin()

1537

In [162]:
deps_idade.iloc[deps_idade['idade'].argmin()]

txNomeParlamentar                      VAL AMÉLIO
nuLegislatura                 2015-01-01 00:00:00
nomeCivil            AUDIVAL AMÉLIO DA SILVA NETO
dataNascimento                1993-12-11 00:00:00
nome                                   VAL AMÉLIO
idade                          7691 days 00:00:00
Name: 1428, dtype: object

### Quem é o deputado mais velho na hora de entrar

In [163]:
deps_idade['idade'].max()

Timedelta('83281 days 00:00:00')

In [164]:
deps_idade['idade'].max()/pd.Timedelta(days=365)

228.16712328767125

Wow! Acho que aqui temos um deputado imortal!

In [165]:
deps_idade.iloc[deps_idade['idade'].argmax()]

txNomeParlamentar                     COSTA FERREIRA
nuLegislatura                    2007-01-01 00:00:00
nomeCivil            ANTÔNIO PEDRO DA COSTA FERREIRA
dataNascimento                   1778-12-26 00:00:00
nome                                  COSTA FERREIRA
idade                            83281 days 00:00:00
Name: 1148, dtype: object

In [167]:
deps_info[deps_info['nome'] == 'COSTA FERREIRA']

,uri,nome,idLegislaturaInicial,idLegislaturaFinal,nomeCivil,cpf,siglaSexo,urlRedeSocial,urlWebsite,dataNascimento,dataFalecimento,ufNascimento,municipioNascimento
138,https://dadosabertos.camara.leg.br/api/v2/depu...,COSTA FERREIRA,2,3,ANTÔNIO PEDRO DA COSTA FERREIRA,NaN,M,NaN,NaN,1778-12-26,1860-07-18,MA,Alcântara
3320,https://dadosabertos.camara.leg.br/api/v2/depu...,COSTA FERREIRA,48,54,ANTONIO DA CONCEIÇÃO COSTA FERREIRA,NaN,M,NaN,NaN,1939-07-28,NaN,MA,Guimarães


In [172]:
deps[deps['txNomeParlamentar'] == 'COSTA FERREIRA']

,txNomeParlamentar,cpf,ideCadastro,nuCarteiraParlamentar,nuLegislatura,sgUF,sgPartido,codLegislatura,numSubCota,txtDescricao,...,numAno,numParcela,txtPassageiro,txtTrecho,numLote,numRessarcimento,vlrRestituicao,nuDeputadoId,ideDocumento,urlDocumento
896359,COSTA FERREIRA,448532387.0,74016.0,571.0,2007,MA,PSC,53,3,COMBUSTÍVEIS E LUBRIFICANTES.,...,2011,0,NaN,NaN,515890,NaN,NaN,59,1996214,NaN
896360,COSTA FERREIRA,448532387.0,74016.0,571.0,2007,MA,PSC,53,5,DIVULGAÇÃO DA ATIVIDADE PARLAMENTAR.,...,2011,0,NaN,NaN,515891,NaN,NaN,59,2012975,NaN
896361,COSTA FERREIRA,448532387.0,74016.0,571.0,2007,MA,PSC,53,5,DIVULGAÇÃO DA ATIVIDADE PARLAMENTAR.,...,2011,0,NaN,NaN,515891,NaN,NaN,59,1996278,NaN
896362,COSTA FERREIRA,448532387.0,74016.0,571.0,2007,MA,PSC,53,10,TELEFONIA,...,2011,0,NaN,NaN,516148,NaN,NaN,59,1997318,NaN
896363,COSTA FERREIRA,448532387.0,74016.0,571.0,2007,MA,PSC,53,10,TELEFONIA,...,2011,0,NaN,NaN,0,0.0,NaN,59,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3289343,COSTA FERREIRA,448532387.0,74016.0,571.0,2011,MA,PSC,54,999,PASSAGEM AÉREA - RPA,...,2013,0,EZEQUIAS MR SOUZA,SLZ/BSB,0,0.0,NaN,59,881079,NaN
3289344,COSTA FERREIRA,448532387.0,74016.0,571.0,2011,MA,PSC,54,999,PASSAGEM AÉREA - RPA,...,2013,0,COSTA FERREIRA,BSB/BSB,0,0.0,NaN,59,880465,NaN
3289345,COSTA FERREIRA,448532387.0,74016.0,571.0,2011,MA,PSC,54,999,PASSAGEM AÉREA - RPA,...,2013,0,COSTA FERREIRA,BSB/BSB,0,0.0,NaN,59,880469,NaN
3289346,COSTA FERREIRA,448532387.0,74016.0,571.0,2011,MA,PSC,54,999,PASSAGEM AÉREA - RPA,...,2013,0,COSTA FERREIRA,BSB/BSB,0,0.0,NaN,59,880475,NaN


In [170]:
deps_info[deps_info['nome'].duplicated()]

,uri,nome,idLegislaturaInicial,idLegislaturaFinal,nomeCivil,cpf,siglaSexo,urlRedeSocial,urlWebsite,dataNascimento,dataFalecimento,ufNascimento,municipioNascimento
168,https://dadosabertos.camara.leg.br/api/v2/depu...,PIRES FERREIRA,2,2,GERVÁSIO PIRES FERREIRA,NaN,M,NaN,NaN,1765-06-26,1838-03-09,PE,Recife
169,https://dadosabertos.camara.leg.br/api/v2/depu...,PEREIRA DE BRITO,2,2,FRANCISCO XAVIER PEREIRA DE BRITO,NaN,M,NaN,NaN,1786-11-17,1844-07-31,PE,Recife
200,https://dadosabertos.camara.leg.br/api/v2/depu...,GALVÃO,3,3,INNOCÊNCIO JOSÉ GALVÃO,NaN,M,NaN,NaN,NaN,NaN,NaN,NaN
230,https://dadosabertos.camara.leg.br/api/v2/depu...,MACIEL MONTEIRO,3,9,ANTONIO PEREGRINO MACIEL MONTEIRO,NaN,M,NaN,NaN,1804-04-30,1868-01-05,PE,Recife
263,https://dadosabertos.camara.leg.br/api/v2/depu...,PEDREIRA,4,4,JOÃO PEDREIRA DO COUTO,NaN,M,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7263,https://dadosabertos.camara.leg.br/api/v2/depu...,SÉRGIO REIS,55,55,SERGIO BAVINI,NaN,M,NaN,NaN,1940-06-23,NaN,SP,São Paulo
7353,https://dadosabertos.camara.leg.br/api/v2/depu...,HÉLIO COSTA,56,56,HÉLIO FRANCISCO DA COSTA,NaN,M,"heliocostasc, heliocostasc",NaN,1954-03-03,NaN,SC,Florianópolis
7549,https://dadosabertos.camara.leg.br/api/v2/depu...,JOÃO ROMA,56,56,JOÃO INÁCIO RIBEIRO ROMA NETO,NaN,M,NaN,NaN,1972-11-17,NaN,PE,Recife
7573,https://dadosabertos.camara.leg.br/api/v2/depu...,DR. JOÃO,56,56,JOÃO ANTONIO HOLANDA CALDAS,NaN,M,NaN,NaN,1990-03-21,NaN,AL,Maceió


Temos 347 nomes repetidos que entraram erradamente no dataset `deps_idade`

In [171]:
deps_info[deps_info['nome'] == 'DR. JOÃO']

,uri,nome,idLegislaturaInicial,idLegislaturaFinal,nomeCivil,cpf,siglaSexo,urlRedeSocial,urlWebsite,dataNascimento,dataFalecimento,ufNascimento,municipioNascimento
7212,https://dadosabertos.camara.leg.br/api/v2/depu...,DR. JOÃO,55,55,JOAO FERREIRA NETO,NaN,M,NaN,NaN,1950-08-05,NaN,CE,Fortaleza
7573,https://dadosabertos.camara.leg.br/api/v2/depu...,DR. JOÃO,56,56,JOÃO ANTONIO HOLANDA CALDAS,NaN,M,NaN,NaN,1990-03-21,NaN,AL,Maceió


**Deixo resolver esse problema para voces! **

### Qual deputado viajou mais do que todo mundo

In [184]:
passagem = deps[(deps['txtDescricao'].str.contains("PASS")) | (deps['txtDescricao'].str.contains("LOCA"))  & (deps['vlrLiquido'] > 0)]

In [185]:
passagem.shape

(1242526, 31)

In [187]:
passagem.groupby('txNomeParlamentar').count()['cpf'].sort_values()

txNomeParlamentar
PROFESSOR LUIZINHO         0
LIDERANÇA DO PT            0
LIDERANÇA DO PSL           0
LIDERANÇA DO PSDB          0
LIDERANÇA DO PODEMOS       0
                        ... 
FELIPE BORNIER          5513
HENRIQUE FONTANA        5915
BONIFÁCIO DE ANDRADA    6828
JÔ MORAES               6908
JORGE TADEU MUDALEN     6963
Name: cpf, Length: 1459, dtype: int64

In [188]:
deps_info[deps_info['nome'] == 'JORGE TADEU MUDALEN']

,uri,nome,idLegislaturaInicial,idLegislaturaFinal,nomeCivil,cpf,siglaSexo,urlRedeSocial,urlWebsite,dataNascimento,dataFalecimento,ufNascimento,municipioNascimento
2906,https://dadosabertos.camara.leg.br/api/v2/depu...,JORGE TADEU MUDALEN,49,55,JORGE TADEU MUDALEN,NaN,M,NaN,NaN,1955-01-03,NaN,SP,Guarulhos


Para onde ele viaja tanto?

In [192]:
passagem[passagem['txNomeParlamentar'] == 'JORGE TADEU MUDALEN']['txtTrecho'].value_counts()

BSB/CGH                 248
BSB/BSB                 195
CGH/BSB                 187
GRU/BSB                 158
BSB/GRU                 136
                       ... 
CWB/GRU                   1
GRU/THE                   1
BSB/REC/BSB               1
BSB/XGRU/JFK/GRU/CGH      1
SSA/CGH                   1
Name: txtTrecho, Length: 130, dtype: int64

Para a casa!


In [195]:
px.bar(x = passagem[passagem['txNomeParlamentar'] == 'JORGE TADEU MUDALEN']['txtTrecho'].value_counts().index, 
       y=passagem[passagem['txNomeParlamentar'] == 'JORGE TADEU MUDALEN']['txtTrecho'].value_counts())

In [201]:
passagem[passagem['txNomeParlamentar'] == 'JORGE TADEU MUDALEN']['sgUF'].iloc[0]

'SP'

In [196]:
deps_info[deps_info['nome'] == 'JÔ MORAES']


,uri,nome,idLegislaturaInicial,idLegislaturaFinal,nomeCivil,cpf,siglaSexo,urlRedeSocial,urlWebsite,dataNascimento,dataFalecimento,ufNascimento,municipioNascimento
6736,https://dadosabertos.camara.leg.br/api/v2/depu...,JÔ MORAES,53,55,MARIA DO SOCORRO JÔ MORAES,NaN,F,NaN,NaN,1946-08-09,NaN,PB,Cabedelo


In [197]:
px.bar(x = passagem[passagem['txNomeParlamentar'] == 'JÔ MORAES']['txtTrecho'].value_counts().index, 
       y=passagem[passagem['txNomeParlamentar'] == 'JÔ MORAES']['txtTrecho'].value_counts())

In [200]:
passagem[passagem['txNomeParlamentar'] == 'JÔ MORAES']['sgUF'].iloc[0]

'MG'